In [14]:
import numpy as np
import pickle
import pandas as pd
import joblib
from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split

In [16]:
def get_Xy_full(df):
    X = df.drop(columns=['TARGET'])
    y = df.TARGET
    return X, y

In [11]:
# load the model
model = joblib.load('model/best_LGB_10k_Undersampled_BestParams.pkl')

#load the X_train dataframe
df = pd.read_pickle('../data/cleaned/client_data_api_dashboard.pkl')

In [12]:
df.shape

(307511, 165)

In [29]:
df.head(5)

,PA_PrLCash_SK_DPD_DEF__mean__mean,PA_DAYS_FIRST_DUE__min,PA_DAYS_LAST_DUE_1ST_VERSION__min,PA_PrLI_DELAY_DAYS_INSTALMENT__max__max,OBS_60_CNT_SOCIAL_CIRCLE,Bur_CB_DAYS_CREDIT_ENDDATE__max,DAYS_EMPLOYED_PERC,Bur_CB_AMT_CREDIT_SUM_DEBT__sum,PA_DAYS_FIRST_DUE__max,Bur_CB_AMT_CREDIT_SUM__mean,...,ORGANIZATION_TYPE_Tradetype7,ORGANIZATION_TYPE_Transporttype1,ORGANIZATION_TYPE_Transporttype2,ORGANIZATION_TYPE_Transporttype3,ORGANIZATION_TYPE_Transporttype4,ORGANIZATION_TYPE_University,ORGANIZATION_TYPE_XNA,TARGET,PREDICTION,PROBA_DEFAULT
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100002,0.0,-565.0,125.0,-12.0,2.0,780.0,0.067329,245781.0,-565.0,108131.945625,...,0,0,0,0,0,0,0,1,1,0.845645
100003,0.0,-2310.0,-1980.0,-1.0,1.0,1216.0,0.070862,0.0,-716.0,254350.125000,...,0,0,0,0,0,0,0,0,0,0.175487
100004,0.0,-784.0,-694.0,-3.0,0.0,-382.0,0.011814,0.0,-784.0,94518.900000,...,0,0,0,0,0,0,0,0,0,0.283380
100006,0.0,-545.0,-215.0,-1.0,2.0,0.0,0.159905,0.0,365243.0,0.000000,...,0,0,0,0,0,0,0,0,0,0.263360
100007,0.0,-2326.0,-2056.0,12.0,0.0,-783.0,0.152418,0.0,-344.0,146250.000000,...,0,0,0,0,0,0,0,0,1,0.515676


In [17]:
# run model predictions on slices of the data, look at false positives and true negatives for each slice

# run full predictions first, get perc default chance for each 

X,y = get_Xy_full(df)
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, stratify=y, random_state=1234)


In [18]:
predictions = model.predict(X)

In [20]:
df['PREDICTION'] = predictions

In [21]:
predictions_proba = model.predict_proba(X)

In [22]:
predictions_proba

array([[0.15435502, 0.84564498],
       [0.82451307, 0.17548693],
       [0.71662011, 0.28337989],
       ...,
       [0.49311504, 0.50688496],
       [0.71619908, 0.28380092],
       [0.44905621, 0.55094379]])

In [25]:
df['PROBA_DEFAULT'] = predictions_proba[:,1]

In [35]:
df_slice = df[['TARGET','PREDICTION','PROBA_DEFAULT']].sort_values('PROBA_DEFAULT')
df_slice.head(5)

,TARGET,PREDICTION,PROBA_DEFAULT
SK_ID_CURR,,,
125223,0,0,0.016557
300909,0,0,0.021071
188857,0,0,0.021887
369594,0,0,0.022088
141405,0,0,0.024176


In [71]:
df_slice[df_slice.PROBA_DEFAULT.between(0.50,0.53)].PREDICTION.sum()

12245

In [56]:
pc = []
fp = []
tn = []

for i in range(0,100,5):
    start_perc = i/100
    end_perc = start_perc+0.05
    slice_between = df_slice[df_slice.PROBA_DEFAULT.between(start_perc,end_perc)]
    print ('SHAPE',slice_between.shape)

    print('FP',slice_between[(slice_between.TARGET == 0) & (slice_between.PREDICTION == 1)].shape[0])
    print('TN',slice_between[(slice_between.TARGET == 1) & (slice_between.PREDICTION == 0)].shape[0])
    pc.append(start_perc)
    fp.append(slice_between[(slice_between.TARGET == 0) & (slice_between.PREDICTION == 1)].shape[0])
    tn.append(slice_between[(slice_between.TARGET == 1) & (slice_between.PREDICTION == 0)].shape[0])
    

SHAPE (486, 3)
FP 0
TN 1
SHAPE (8120, 3)
FP 0
TN 60
SHAPE (18537, 3)
FP 0
TN 217
SHAPE (24645, 3)
FP 0
TN 431
SHAPE (27020, 3)
FP 0
TN 604
SHAPE (27454, 3)
FP 0
TN 819
SHAPE (26752, 3)
FP 0
TN 1027
SHAPE (25578, 3)
FP 0
TN 1230
SHAPE (23566, 3)
FP 0
TN 1396
SHAPE (22057, 3)
FP 0
TN 1726
SHAPE (20195, 3)
FP 18375
TN 0
SHAPE (17882, 3)
FP 15909
TN 0
SHAPE (15954, 3)
FP 13865
TN 0
SHAPE (14133, 3)
FP 11857
TN 0
SHAPE (11714, 3)
FP 9437
TN 0
SHAPE (9498, 3)
FP 7321
TN 0
SHAPE (7381, 3)
FP 5220
TN 0
SHAPE (4734, 3)
FP 3066
TN 0
SHAPE (1717, 3)
FP 903
TN 0
SHAPE (88, 3)
FP 29
TN 0


In [60]:
df_display = pd.DataFrame()
df_display['pc'] = pc
df_display['fp'] = fp
df_display['tn'] = tn
df_display


,pc,fp,tn
0,0.00,0,1
1,0.05,0,60
2,0.10,0,217
3,0.15,0,431
4,0.20,0,604
5,0.25,0,819
6,0.30,0,1027
7,0.35,0,1230
8,0.40,0,1396
9,0.45,0,1726
